In [5]:
import numpy as np
import pandas as pd
import warnings
import os
import sys
import math
import copy
warnings.filterwarnings('ignore')

class UserCF:
    def __init__(self):
        pass
    
    def load_data(self):
        path1 = 'movielens_data/users.dat'
        path2 = 'movielens_data/ratings.dat'
        path3 = 'movielens_data/movies.dat'
        
        users = pd.read_csv(path1, header = None, sep = '::')
        ratings = pd.read_csv(path2, header = None, sep = '::')
        movies = pd.read_csv(path3, header = None, sep = '::')
        
        users.columns = ['user_id', 'gender', 'age', 'occupation', 'zip_code']
        ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
        movies.columns = ['movie_id', 'title', 'genres']
        
        return users, ratings, movies
    
    def split_train_test(self, users, ratings):
        all_data = ratings.merge(users, on = 'user_id', how = 'left')
        
        shuffled_indices = np.random.permutation(len(all_data))
        test_set_size = int(len(all_data) * 0.2)
        
        test_indices = shuffled_indices[:test_set_size]
        train_indices = shuffled_indices[test_set_size:]
        
        return all_data.iloc[train_indices], all_data.iloc[test_indices]
    
    def user_similarity(self, train_data):
        
        # build inverse table for item_users
        print('establising the inverse table...')
        self.item_users = dict()
        for index, row in train_data.iterrows():
            if row['movie_id'] not in self.item_users:
                self.item_users[row['movie_id']] = set()
            self.item_users[row['movie_id']].add(row['user_id'])
            
        # calculate co-rated items between users
        print('calculating the co-rated matrix...')
        self.user_list = list(set(train_data['user_id']))
        movie_list_len = len(set(train_data['movie_id']))
        
        C = [[0 for i in range(len(self.user_list))] for i in range(len(self.user_list))]
        W = copy.deepcopy(C)
        
        self.N = dict()
        i = 1
        for movie, users in self.item_users.items():
            #print(str(i) + ' in ' + str(movie_list_len))
            i += 1
            for u in users:
                if self.user_list.index(u) not in self.N:
                    self.N[self.user_list.index(u)] = 0
                self.N[self.user_list.index(u)] += 1
                for v in users:
                    if u == v:
                        continue
                    #C[self.user_list.index(u)][self.user_list.index(v)] += 1
                    C[self.user_list.index(u)][self.user_list.index(v)] += 1 / math.log(1 + len(users))
                    
        # calculate finial similarity matrix W
        print('calculating the final similarity matrix...')
        for i in range(len(C)):
            for j in range(len(C)):
                if i == j or C[i][j] == 0:
                    continue
                W[i][j] = C[i][j] / math.sqrt(self.N[i] * self.N[j])
        
        return W
    
    def recommend(self, movies, ratings, user, W, k=5, threshold=3):
        # find the first k highest similarity values
        similar = sorted(enumerate(W[self.user_list.index(user)]), reverse=True, key=lambda x:x[1])[0:k]
        
        first_k = []
        similar_users = []
        for s in similar:
            first_k.append(s[1])
            similar_users.append(s[0])
        #print(first_k)
        
        if first_k[0] == 0:
            return []
    
    
        recommend_score = dict()
        movie_list = list(movies['movie_id'])
    
        # check all the products,
        # if current user hasn't touch the product,
        # and his 'similar users' have touched the product,
        # and gave an evluation more than the threshold,
        # then recommend it to the current user
        for movie in movie_list:
            if movie in self.item_users:
                if user not in self.item_users[movie]:    # if user didn't interact with the product
                    i = 0
                    for u in similar_users:
                        if u in self.item_users[movie]:    # if the similar users have interacted with the product
                            rating = int(ratings.loc[(ratings['user_id'] == u) & (ratings['movie_id'] == movie)]['rating'])
                            if rating >= threshold:
                                if movie not in recommend_score:
                                    recommend_score[movie] = 0
                                recommend_score[movie] += first_k[i] * rating     
                        i += 1
            #sys.exit()
                        
        return recommend_score

In [2]:
users, ratings, movies = UserCF().load_data()
print('data loaded')

data loaded


In [6]:
ucf = UserCF()
train_data, test_data = ucf.split_train_test(users, ratings)
W = ucf.user_similarity(train_data.head(int(train_data.shape[0]/100)))
print('done')

establising the inverse table...
calculating the co-rated matrix...
calculating the final similarity matrix...
done


In [15]:
test_user = list(set(test_data['user_id']))
for user in test_user:
    if user not in ucf.user_list:
        continue
    result = ucf.recommend(movies, ratings, user, W, k=3, threshold=5)
    if len(result) != 0:
        print('user_id: ', user)
        for movie, score in result.items():
            print(movie, movies.loc[movies['movie_id'] == movie]['title'].values[0], score)
        print('==================================================')

user_id:  3
1203 12 Angry Men (1957) 1.894615908449756
user_id:  10
3578 Gladiator (2000) 1.793639458624552
user_id:  13
2 Jumanji (1995) 1.5346383821506744
user_id:  17
908 North by Northwest (1959) 3.2181816482491765
user_id:  19
3107 Backdraft (1991) 1.161714202098663
user_id:  23
912 Casablanca (1942) 1.3952765663781181
user_id:  24
1801 Man in the Iron Mask, The (1998) 1.5354628659284384
2001 Lethal Weapon 2 (1989) 1.5354628659284384
1196 Star Wars: Episode V - The Empire Strikes Back (1980) 2.1714724095162588
919 Wizard of Oz, The (1939) 1.5354628659284384
user_id:  28
3911 Best in Show (2000) 1.4228035110431094
user_id:  33
608 Fargo (1996) 0.9084525068254421
user_id:  35
2028 Saving Private Ryan (1998) 1.5533373363990297
user_id:  38
3578 Gladiator (2000) 1.6129820911147803
user_id:  42
595 Beauty and the Beast (1991) 2.5694917118487535
2747 Little Shop of Horrors, The (1960) 1.8169050136508842
2181 Marnie (1964) 1.8169050136508842
user_id:  53
1961 Rain Man (1988) 1.0489905987

user_id:  225
296 Pulp Fiction (1994) 0.932512776043915
user_id:  230
1217 Ran (1985) 2.5503486164919735
1674 Witness (1985) 1.8169050136508842
user_id:  231
904 Rear Window (1954) 1.1392385458963108
user_id:  234
1060 Swingers (1996) 1.6175772657433622
1094 Crying Game, The (1992) 1.6175772657433622
user_id:  235
1356 Star Trek: First Contact (1996) 1.9732190434332983
user_id:  236
3893 Nurse Betty (2000) 1.6090908241245883
user_id:  238
771 Vie est belle, La (Life is Rosey) (1987) 1.8169050136508842
user_id:  240
1296 Room with a View, A (1986) 1.594644944519005
858 Godfather, The (1972) 1.594644944519005
user_id:  242
2139 Secret of NIMH, The (1982) 1.0857362047581294
2334 Siege, The (1998) 1.2751743082459868
user_id:  245
1265 Groundhog Day (1993) 0.8564972372829178
2324 Life Is Beautiful (La Vita � bella) (1997) 0.8564972372829178
2617 Mummy, The (1999) 0.8564972372829178
user_id:  249
1965 Repo Man (1984) 1.894615908449756
user_id:  252
898 Philadelphia Story, The (1940) 1.517065

user_id:  416
2791 Airplane! (1980) 1.9732190434332983
user_id:  419
671 Mystery Science Theater 3000: The Movie (1996) 1.2022458674074696
user_id:  424
2797 Big (1988) 1.4392145858854952
user_id:  426
1185 My Left Foot (1989) 1.4744321597526358
1226 Quiet Man, The (1952) 1.4744321597526358
2019 Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954) 2.5503486164919735
user_id:  429
356 Forrest Gump (1994) 1.3784030065341186
user_id:  433
1267 Manchurian Candidate, The (1962) 3.6067376022224087
2791 Airplane! (1980) 3.6067376022224087
user_id:  438
912 Casablanca (1942) 1.1392385458963108
user_id:  441
903 Vertigo (1958) 1.4834967315163903
user_id:  443
1653 Gattaca (1997) 1.8463468653442752
user_id:  445
930 Notorious (1946) 1.1405505745597189
user_id:  453
2599 Election (1999) 0.8824503831227931
user_id:  457
1018 That Darn Cat! (1965) 1.4834967315163903
372 Reality Bites (1994) 1.4834967315163903
2399 Santa Claus: The Movie (1985) 1.4834967315163903
user_id:  461
2058 Ne

user_id:  617
3396 Muppet Movie, The (1979) 2.7905531327562363
user_id:  631
213 Burnt By the Sun (Utomlyonnye solntsem) (1994) 2.275598066567093
user_id:  637
2268 Few Good Men, A (1992) 1.2038688172479102
user_id:  639
3418 Thelma & Louise (1991) 1.3138171562999033
user_id:  643
2791 Airplane! (1980) 1.1392385458963108
user_id:  645
589 Terminator 2: Judgment Day (1991) 0.947307954224878
user_id:  647
2804 Christmas Story, A (1983) 1.5533373363990297
1221 Godfather: Part II, The (1974) 1.5533373363990297
user_id:  648
1248 Touch of Evil (1958) 1.2682946241971886
596 Pinocchio (1940) 1.2682946241971886
user_id:  651
1299 Killing Fields, The (1984) 0.9711119837887232
318 Shawshank Redemption, The (1994) 0.9711119837887232
user_id:  654
750 Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963) 2.275598066567093
user_id:  659
1653 Gattaca (1997) 1.8463468653442752
user_id:  660
1641 Full Monty, The (1997) 1.3952765663781181
1805 Wild Things (1998) 1.3952765663781181

user_id:  835
1210 Star Wars: Episode VI - Return of the Jedi (1983) 2.035356786524445
user_id:  837
1185 My Left Foot (1989) 2.5694917118487535
1345 Carrie (1976) 1.2847458559243767
user_id:  839
750 Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963) 0.9290090204588879
user_id:  853
296 Pulp Fiction (1994) 1.4228035110431094
3897 Almost Famous (2000) 1.4228035110431094
user_id:  855
1248 Touch of Evil (1958) 2.1967507280760064
908 North by Northwest (1959) 0.9824167915200211
2918 Ferris Bueller's Day Off (1986) 0.9824167915200211
user_id:  857
3104 Midnight Run (1988) 3.1066746727980594
994 Big Night (1996) 3.1066746727980594
user_id:  858
1288 This Is Spinal Tap (1984) 1.0355582242660197
908 North by Northwest (1959) 1.0355582242660197
2150 Gods Must Be Crazy, The (1980) 1.0355582242660197
user_id:  864
1584 Contact (1997) 0.9016844005556022
3100 River Runs Through It, A (1992) 0.9016844005556022
1678 Joy Luck Club, The (1993) 0.9016844005556022
user_id:  880

...

user_id:  5223
592 Batman (1989) 1.3882339503596355
user_id:  5227
260 Star Wars: Episode IV - A New Hope (1977) 1.2537001801720573
user_id:  5232
2940 Gilda (1946) 1.6111266023847666
2791 Airplane! (1980) 1.6111266023847666
user_id:  5233
549 Thirty-Two Short Films About Glenn Gould (1993) 3.2181816482491765
user_id:  5234
3186 Girl, Interrupted (1999) 1.700232410994649
user_id:  5239
3538 East is East (1999) 1.3882339503596355
user_id:  5241
1060 Swingers (1996) 1.6175772657433622
1094 Crying Game, The (1992) 1.6175772657433622
user_id:  5248
25 Leaving Las Vegas (1995) 2.275598066567093
586 Home Alone (1990) 2.275598066567093
1377 Batman Returns (1992) 2.275598066567093
user_id:  5250
282 Nell (1994) 1.0489905987233188
user_id:  5253
1610 Hunt for Red October, The (1990) 1.6111266023847666
908 North by Northwest (1959) 1.2479732999335877
user_id:  5254
593 Silence of the Lambs, The (1991) 1.6090908241245883
1954 Rocky (1976) 1.6090908241245883
1206 Clockwork Orange, A (1971) 1.60909

user_id:  5391
1207 To Kill a Mockingbird (1962) 1.2537001801720573
user_id:  5394
317 Santa Clause, The (1994) 0.9366976577610266
user_id:  5395
2336 Elizabeth (1998) 1.8033688011112043
user_id:  5396
2947 Goldfinger (1964) 1.3784030065341186
user_id:  5401
1345 Carrie (1976) 1.2478883432304582
3490 Horror Express (1972) 1.2478883432304582
3851 I'm the One That I Want (2000) 1.2478883432304582
1358 Sling Blade (1996) 3.2181816482491765
903 Vertigo (1958) 1.2478883432304582
3608 Pee-wee's Big Adventure (1985) 1.2478883432304582
231 Dumb & Dumber (1994) 3.2181816482491765
2606 Idle Hands (1999) 1.2478883432304582
user_id:  5403
19 Ace Ventura: When Nature Calls (1995) 1.4834967315163903
1485 Liar Liar (1997) 1.4834967315163903
user_id:  5406
1304 Butch Cassidy and the Sundance Kid (1969) 1.4392145858854952
1617 L.A. Confidential (1997) 1.0394968327903782
user_id:  5413
1967 Labyrinth (1986) 1.0857362047581294
user_id:  5418
593 Silence of the Lambs, The (1991) 1.1392385458963108
965 39 

user_id:  5600
2857 Yellow Submarine (1968) 0.9816296401726872
2918 Ferris Bueller's Day Off (1986) 1.3882339503596355
user_id:  5603
213 Burnt By the Sun (Utomlyonnye solntsem) (1994) 1.6090908241245883
user_id:  5604
1788 Men With Guns (1997) 1.4724444602590308
user_id:  5607
3793 X-Men (2000) 2.012148021909223
user_id:  5608
1060 Swingers (1996) 1.6175772657433622
1094 Crying Game, The (1992) 1.6175772657433622
user_id:  5612
288 Natural Born Killers (1994) 1.8033688011112043
2959 Fight Club (1999) 1.8033688011112043
user_id:  5615
3629 Gold Rush, The (1925) 1.5354628659284384
951 His Girl Friday (1940) 1.5354628659284384
user_id:  5618
2920 Children of Paradise (Les enfants du paradis) (1945) 1.2038688172479102
1617 L.A. Confidential (1997) 1.4834967315163903
3814 Love and Death (1975) 1.2038688172479102
user_id:  5620
1224 Henry V (1989) 0.8501162054973245
356 Forrest Gump (1994) 0.8501162054973245
1079 Fish Called Wanda, A (1988) 1.0983753640380032
user_id:  5621
1197 Princess Br

user_id:  5786
1285 Heathers (1989) 0.896819729312276
user_id:  5787
1262 Great Escape, The (1963) 0.9102392266268373
3175 Galaxy Quest (1999) 0.812544623836712
user_id:  5789
1617 L.A. Confidential (1997) 1.2479732999335877
1965 Repo Man (1984) 1.2479732999335877
user_id:  5798
2858 American Beauty (1999) 1.8033688011112043
1299 Killing Fields, The (1984) 1.5354628659284384
318 Shawshank Redemption, The (1994) 1.5354628659284384
user_id:  5801
1371 Star Trek: The Motion Picture (1979) 2.1967507280760064
2300 Producers, The (1968) 2.1967507280760064
735 Cemetery Man (Dellamorte Dellamore) (1994) 2.1967507280760064
user_id:  5802
2916 Total Recall (1990) 3.6067376022224087
user_id:  5811
2723 Mystery Men (1999) 0.9703182758590927
user_id:  5812
2396 Shakespeare in Love (1998) 0.8758781041999354
user_id:  5813
1304 Butch Cassidy and the Sundance Kid (1969) 1.2478883432304582
1086 Dial M for Murder (1954) 1.6090908241245883
223 Clerks (1994) 1.6090908241245883
user_id:  5815
356 Forrest G

user_id:  6006
1079 Fish Called Wanda, A (1988) 2.012148021909223
user_id:  6011
1888 Hope Floats (1998) 2.275598066567093
user_id:  6015
1035 Sound of Music, The (1965) 1.7647806193238058
2413 Clue (1985) 1.7647806193238058
user_id:  6017
1097 E.T. the Extra-Terrestrial (1982) 1.6175772657433622
1214 Alien (1979) 1.6175772657433622
3081 Sleepy Hollow (1999) 1.6175772657433622
user_id:  6021
288 Natural Born Killers (1994) 1.8463468653442752
1537 Shall We Dance? (Shall We Dansu?) (1996) 1.065988859723939
509 Piano, The (1993) 1.065988859723939
318 Shawshank Redemption, The (1994) 1.8463468653442752
user_id:  6023
3160 Magnolia (1999) 1.0983753640380032
926 All About Eve (1950) 1.0983753640380032
user_id:  6029
597 Pretty Woman (1990) 1.894615908449756
2501 October Sky (1999) 1.3396957566087337
user_id:  6035
592 Batman (1989) 0.8214559901046841
1231 Right Stuff, The (1983) 0.8214559901046841
223 Clerks (1994) 0.8214559901046841


In [18]:
(movies.loc[movies['movie_id'] == 3246]['title'].index[0])

3177

In [22]:
movies.iloc[3177].title
#movies.head()

'Malcolm X (1992)'

In [334]:
x = ratings.loc[(ratings['user_id'] == 321) & (ratings['movie_id'] == 2282)]['rating']
if x >= 1:
    print('x')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [6]:
a = [3,4,1,7,2]
b=sorted(enumerate(a), key=lambda x:x[1], reverse=True)
b

[(3, 7), (1, 4), (0, 3), (4, 2), (2, 1)]

In [43]:
b[0]

(2, 1)

In [ ]:
2834 2959

In [187]:
a = [0,1,2]
b = [a] * len(a)
for i in range(len(b)):
    for j in range(len(b)):
        if i == j:
            continue
        b[i][j] += 3
b

[[6, 7, 8], [6, 7, 8], [6, 7, 8]]

In [59]:
a = [0,1,2]
b = [[0 for i in range(len(a))] for i in range(len(a))]
for i in range(len(b)):
    for j in range(len(b)):
        if i == j:
            continue
        b[i][j] += 3
b

[[0, 3, 3], [3, 0, 3], [3, 3, 0]]

In [61]:
b[0].sort(reverse=True)

In [64]:
b[0]

[3, 3, 0]

In [131]:
a = [2,3,3,1,1]
a.index(1)

3

In [8]:
math.log(2) + math.log(5)

2.3025850929940455

In [9]:
math.log(10)

2.302585092994046